In [49]:
import pykx as kx
import pandas as pd
import numpy as np
import plotly.express as px
# import matplotlib.pyplot as plt
import datetime
import pytz


## define connection to Gateway
gw = kx.QConnection(host='localhost', port=49226, no_ctx = True)

END_TIME = datetime.datetime.now(tz=pytz.utc)                                       ## Now
START_TIME = END_TIME - datetime.timedelta(minutes = 5000)                           ## 300 Mins ago

## Query 1
query_params = {
    'table': 'trade',
    'startTS': START_TIME,
    'endTS': END_TIME,
    'sortCols': 'time',
    'filter': [[b'in','sym','IBM']]
}

empty_dict = {'':''}

trade = gw(kx.SymbolAtom('.kxi.getData'), query_params, 'f', empty_dict)[1]

## Query 2
query_params = {
    'table': 'quote',
    'startTS': START_TIME,
    'endTS': END_TIME,
    'sortCols': 'time',
    'filter': [[b'in','sym','IBM']]
}

quote = gw(kx.SymbolAtom('.kxi.getData'), query_params, 'f', empty_dict)[1]

tab = kx.q.aj(np.array(['sym', 'time']), trade, quote)

# px.line(df, x='time', y=['price','bid','ask']).show() 
# px.line(df, x='time', y='price').show() 

# kx.q('select High:max price, Low:min price, Avg:avg price by 10 xbar time.minute from $1', df)




# print(df)
# print(t.index)

px.line(tab.pd(), x='time', y='price').show() 

# px.line(df, x='Time', y=['High','Low','Avg']).show() 
# px.line(df, x='Time', y=['Open', 'High', 'Low', 'Close']).show() 
######### IAN #########
# containerize jupyter notebook 

# open, high, low, close

# xbar in ANSI SQL - not available until next week
# kx.q.sql('select xbar(10,price) from $1', trade)



######### JACK #########
# trade impact report
# spread before and after a trade


In [50]:
import plotly.graph_objects as go


res = kx.q.qsql.select(
                table = tab, 
                columns = {'Open':'first price', 'High':'max price', 'Low':'min price', 'Close':'last price'},
                by = {'Time': ['.q.xbar', 1, 'time.minute']}
                )

kx.q('.util.unkey:{0!x}')
unkey_res = kx.q.util.unkey(res)

df = unkey_res.pd()

fig = go.Figure(data=go.Ohlc(x=df['Time'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']))
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()